In [1]:
import pandas as pd
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from textblob import TextBlob
import wordsegment as ws 
ws.load()
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
data_sports = pd.read_json('tweets_sports.json', lines = True)
data_sports.sort_values('text',inplace=True)
data_sports.drop_duplicates(subset='text',keep = False, inplace=True)
data_sports.shape

(4768, 37)

In [3]:
def clean_tweets(df):
  i = 1  
  all_tweets = []
  tweets = df['text'].values.tolist();
  for text in tweets:
    print()
    print(i)
    i +=1
    print(text)
    text = text.lower()
    text_pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]|))+')
    text = text_pattern.sub('', text)
    emoji = re.compile("["
                        u"\U0001F600-\U0001FFFF"   #EMOJIS
                        u"\U0001F300-\U0001F5FF"   #Symbols and Pictographs
                        u"\U0001F680-\U0001F6FF"   #map symbols
                        u"\U0001F1E0-\U0001F1FF"   #flags
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
    text = emoji.sub(r'', text)
    text = re.sub(r'@[a-z0-9]+','',text)
   # text = re.sub(r'#', '', text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"this's", "this is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", "will", text)
    text = re.sub(r"\'ve", "have", text)
    text = re.sub(r"\'m", "am", text)
    text = re.sub(r"\'d", "would", text)
    text = re.sub(r"\'re", "are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"didn't", "did not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"haven't", "have not", text)
    text = re.sub(r'rt[\s]+', '', text)

    text = re.sub(r"[,.\"!#$%^&*(){}?/;'~:<>+=-]", "", text)
    
    tokens = word_tokenize(text)
    
    text = [ws.clean(w) for w in text if len(ws.clean(w)) > 0]
    text = [ws.segment(w)[0] for w in text]   #segmentation
    
    stop_words = set(stopwords.words("english"))
    stop_words.discard("not")
    table = str.maketrans('','',string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]

    words = [w for w in words if not w in stop_words]
    words = ' '.join(words)
        
    print(words)
    
    all_tweets.append(words)
  return all_tweets

In [4]:
nan_value = float("NaN")

Cleaned_tweets = clean_tweets(data_sports)

data_sports['text_cleaned'] = Cleaned_tweets

data_sports.replace("", nan_value, inplace=True)
data_sports.dropna(subset=['text_cleaned'], inplace=True)

data_sports = data_sports.sample(frac = 1)

data_sports = data_sports[data_sports['text_cleaned'].str.split().str.len().gt(4)]

data_sports = data_sports[['id', 'text_cleaned','text']]

data_sports.to_csv('tweets_cleaned_sports.csv',index=False)


1
"And it's gonna be Bayern Munich for Real Madrid once again, we're in for another European Classic!" https://t.co/ECZ6INpnMd
gon na bayern munich real madrid weare another european classic

2
"But every year we lose important players...Every season we sell players, it’s part of the identity of the club."… https://t.co/lVyyruqr8v
every year lose important playersevery season sell players paof identity

3
"GWADAR CRICKET STADIUM"
One of the most picturesque grounds in Pakistan.
@falamb3 thank you for highlighting this… https://t.co/IyJAuNAscO
gwadar cricket stadium one picturesque grounds pakistan thank highlighting

4
"GWADAR CRICKET STADIUM"
One of the most picturesque grounds in Pakistan.
@falamb3 thank you for highlighting this… https://t.co/fX5pb0PWHa
gwadar cricket stadium one picturesque grounds pakistan thank highlighting

5
"He feels he wants to be at home with his family and we respect that."

#INDvENG

https://t.co/ZmOUpG7z4O
feels wants home family respect indveng

6
"I do

indvsa india finish solid batting performance big put early several partnerships

76
#INDvSA 

WICKET!
A TERRIFIC catch from Sekhukhune ends the partnership and Harmanpreet's innings on 36 off 46 

Is… https://t.co/zYdslyiRjV
indvsa wicket terrific catch sekhukhune ends partnership harmanpreets innings

77
#INDvsEND #RohitSharma #Shardulthakur ...... Yes yes we are back in game to play in front foot..🇮🇳
indvsend rohitsharma shardulthakur yes yes back game play front foot

78
#INDvsEND #suryakumar #stokes #NotOut #thakur #INDvENGt20 

To  shastri,

Temperature is getting down. I am getting… https://t.co/7YfCWBeqSg
indvsend suryakumar stokes notout thakur shastri temperature getting

79
#INDvsENG

Bhuvi🔥 is overshadowed by Bumrah in a same way
 Dravid is Overshadowed by Sachin

Under - rated😐… https://t.co/fOeaMhxwmW
indvseng bhuvi overshadowed bumrah way dravid overshadowed sachin

80
#INDvsENG
#RohitSharma

Rohit Sharma's captaincy for team India in today's match be like : https://t.co

ucldraw bayernliverpool chelseareal madrid psg porto mcitydordmund

183
#UCLdraw  my predictions
Chelsea vs  Porto
Liverpool vs Man city 
Bayern vs Real Madrid 
PSG vs Dortmund
ucldraw predictions chelsea vs porto liverpool vs man city bayern vs real madrid psg vs dortmund

184
#UCLdraw #UCL 

Chelsea vs Liverpool

Bayern vs PSG

Real Madrid vs Borussia Dortmund

Man City vs Porto 

Prediction 😅😅😅
ucldraw ucl chelsea vs liverpool bayern vs psg real madrid vs borussia dortmund man city vs porto prediction

185
#UCLdraw @ChampionsLeague here's the draw
. Chelsea vs Psg
Manchester city vs Liverpool
Realmadrid vs Dortmund
Bayern munich vs Porto
ucldraw heres draw chelsea vs psg manchester city vs liverpool realmadrid vs dortmund bayern munich vs porto

186
#UCLdraw My Bet :

DORTMUND - CHELSEA
MAN. CITY - PORTO
BAYERN - REAL MADRID
PSG - LIVERPOOL
ucldraw bet dortmund chelsea man city porto bayern real madrid psg liverpool

187
#UCLdraw My prediction;

Liverpool vs Manchester City

Porto V

offer million something english tax rice

292
@1uca86 If Grealish and Maddison were fit neither Lingard or JWP would be in the squad. Dier is just dog shit. 🤷🏽‍♂️
grealish maddison fit neither lingard jwp would squad dier dog shit

293
@2014_manel @Goal_India @ritabrata20 Yes , India has a  full of talented players indeed but lack of opportunity is… https://t.co/jjpw0mdwzR
manel india yes india full talented players indeed lack opportunity

294
@7FootballCorner Leicester not in but id want Porto and avoid Bayern.
leicester not id want porto avoid bayern

295
@90min_Football @England Jesse lingard is back❤️💎
football jesse lingard back

296
@AAlabaji @ChampionsLeague Porto is bigger than your club
porto bigger club

297
@AGLeague_EN - Weekly Magazine - Episode 17

#FuteboldasArabias #QuarentenadaBola #AGLeague #TV #Sports #Streaming… https://t.co/WAqxZNO6P6
en weekly magazine episode futeboldasarabias quarentenadabola agleague tv sports

298
@ANI Here's an interesting pie chart depictin

chelsea vs real madrid psg vs liverpool man city vs bayern dortmund vs porto

426
@ChampionsLeague Chelsea vs porto pleaseee
chelsea vs porto pleaseee

427
@ChampionsLeague Chelsea will stop this guy...
Chelsea vrs Dortmund

What a reunion it will be for TT
chelsea stop guy chelsea vrs dortmund reunion tt

428
@ChampionsLeague City Porto 🔥
Chelsea Bayern 🔥
Real Paris 🔥
Dortmund Liverpool  🔥
city porto chelsea bayern real paris dortmund liverpool

429
@ChampionsLeague City v Bayern 

I can feel it
city v bayern feel

430
@ChampionsLeague City vs bayern a good game to watch
city vs bayern good game watch

431
@ChampionsLeague Dortmund-real madrid
dortmundreal madrid

432
@ChampionsLeague Give city Bayern
give city bayern

433
@ChampionsLeague I just want Chelsea or Man City(city fan overall) win this time🔵✔️and I don't want Bayern and Liverpool win once again🔴❌
want chelsea man citycity fan overall win timeand not want bayern liverpool win

434
@ChampionsLeague I think Bayern will win ag

...

lakers trying acquire tucker little not sure houston cares

4202
The Milan - United tie feels a lot like the Real Madrid - Bayern tie from 2018. 

Bayern were the better team and d… https://t.co/QKGgUtc4wp
milan united tie feels lot like real madrid bayern tie bayern better team

4203
The Most Versatile Batsman in The History of The Game.

Just an International Batsman with a range of Indian Fans..… https://t.co/0UDZfvN0xc
versatile batsman history game international batsman range indian

4204
The NBA all of the upcoming Basketball action this weekend:

https://t.co/zsdRGVvRQZ

#NBA 
#Basetball https://t.co/NzmRxAJjNn
nba upcoming basketball action weekend nba basetball

4205
The NFL got it wrong with that wild Patrick Mahomes play but it's OK - For The Win https://t.co/wtIBVIzNoz #NFL… https://t.co/rYff0CUhOb
nfl got wrong wild patrick mahomes play ok win

4206
The No. 1 T20I batsman @dmalan29 is certainly a player to watch out for in IPL 2021 auction 👀

#Cricket #DawidMalan… https://

top ictmin players minimum min kevin de bruyne highest ranked player ictmin

4312
Top daddy 100s by Indian batsmen in Tests: 

Sunil Gavaskar 221 in London
Rahul Dravid 233 in Adelaide
VVS Laxman 2… https://t.co/Bog1WfGFeS
top daddy indian batsmen tests sunil gavaskar london rahul dravid adelaide vvs laxman

4313
Top scores for India when batting first against SA

302/3 - (2018)
248/5- (2021) - Today 👈
240/10- (2018)
213/7- (2… https://t.co/Y9EHfB29Wy
top scores india batting first sa today

4314
Toss jeto
Bowling karo aur
Match jeto

#INDvENG
toss jeto bowling karo aur match jeto indveng

4315
Tottenham go to extra-time with Dinamo Zagreb, Arsenal through, no Liverpool stars in England squad as Lingard reca… https://t.co/ndiLMbHsRu
tottenham go extratime dinamo zagreb arsenal liverpool stars england squad lingard

4316
Tou a sentir Dortmund
tou sentir dortmund

4317
Tour postponement: CSA lodges official complaint against Cricket Australia with ICC #Cricket https://t.co/DnCEmGHlQd
tou

well bowled chepauk indveng indvseng

4434
Well deserved one bruh
Keep rocking
well deserved one bruh keep rocking

4435
Well done @faf1307 , you will be missed by those who played with and against you. 🙌🏻🙌🏻🙌🏻 #Cricket 

@MSAalamOfficial #MSAalam
well done missed played cricket msaalam

4436
Well done rohit sharma ( @ImRo45 )
What a captaincy between 16 to 20 overs 👏👏🙌🙌❤❤ 

#RohitSharma #Captain #BCCI… https://t.co/ZqEA1a5LCz
well done rohit sharma captaincy overs rohitsharma captain

4437
Well played @faf1307. Another top player says goodbye to the South African @OfficialCSA test team. Opens up an oppo… https://t.co/ooFB96kv1o
well played another top player says goodbye south african test team opens

4438
Well played Kasuza👏🏿. You did well brother. 🇿🇼🏏

#Cricket #AFGvZIM
well played kasuza well brother cricket afgvzim

4439
Well played Team India 
Congratulations winners #INDvsENG
well played team india congratulations winners indvseng

4440
Well played skipper! #MithaliRaj
well playe

yall kno tina turnerucldrawkim kardashianmanchester

4552
Yasir Arafat claimed a total of 29 wickets and scored 260 runs across all formats in his international career, whic… https://t.co/gTlrmKWtdb
yasir arafat claimed total wickets scored runs across formats international career

4553
Yatra nari,Aj ki awaj. 
Proud moment for all of us. 
Congratulations.
#India #Cricket #mitaliraj https://t.co/cum94Wq1P6
yatra nariaj ki awaj proud moment us congratulations india cricket mitaliraj

4554
Yes but both these teams for now a giants these days and this bilateral series can be the best series in t20 right… https://t.co/2eKw3JWlaW
yes teams giants days bilateral series best series

4555
Yes yes
yes yes

4556
Yesss Dortmund please 💪💪
yesss dortmund please

4557
Yet another milestone for Mithali Raj who completes 10,000 runs for India in International cricket.

Apart from thi… https://t.co/67gUvw4gaE
yet another milestone mithali raj completes runs india international cricket apafrom

4558
Yet 

not surprised bayern draws porto

4690
ye baaat... perfect reply sir....
ye baaat perfect reply sir

4691
{ #UCLdraw #UCL #Bayern #Dortmund #teampsg #psg #porto #Chelsea #link #lien #livestream }

🏴󠁧󠁢󠁥󠁮󠁧󠁿 Look the draw uc… https://t.co/RmA1pEOYVg
ucldraw ucl bayern dortmund teampsg psg porto chelsea link lien livestream look draw

4692
‘Mein Virar Ka Ladka Hoon,’- Prithvi Shaw Reveals Sachin Tendulkar’s Advice Is The Reason Behind His Scintillating… https://t.co/6jzvYkW5yJ
mein virar ka ladka hoon prithvi shaw reveals sachin tendulkar advice reason behind

4693
‘My NFL brothers continue to die’ | Ryan Leaf responds to death of Vincent Jackson - Atlanta Journal Constitution… https://t.co/qGvxOa6jiK
nfl brothers continue die ryan leaf responds death vincent jackson atlanta journal

4694
‘’@TBanton18’’said that I have Arrived in Pakistan. Looking forward to the start of this years @thePSLt20! See you… https://t.co/pBaeFnAN2V
said arrived pakistan looking forward staof years see

4695
‘’@T